In [57]:
import pandas as pd

orders = pd.read_csv('orders.csv')
orders["Timestamp"] = pd.to_datetime(orders["Timestamp"])
orders.columns = ['timestamp', 'group id', 'side', 'price', 'size']
orders["order id"] = orders.index + 1
orders.head()

,timestamp,group id,side,price,size,order id
0,2024-09-30 20:26:01,5,buy,8.0,50,1
1,2024-09-30 20:26:11,14,sell,11.0,20,2
2,2024-09-30 20:26:14,10,sell,11.0,100,3
3,2024-09-30 20:26:15,12,sell,10.0,50,4
4,2024-09-30 20:26:16,7,buy,5.0,50,5


## Main structure of CLOB
- deal with each order and try to match a successful trade
- place in a proper position if not matched / partly matched

In [87]:
# rename to bid & ask in the final display
clob = {"buy": {}, "sell": {}}
trade = pd.DataFrame(columns=["timestamp", "buy", "sell", "price", "size"]) 

In [10]:
def match_trade(order):
    # return outstanding, matched_trades
    matched_trades = []

    side, price, outstanding = order["side"], float(order["price"]), int(order["size"])
    opp_side = "buy" if side == "sell" else "sell"
    opp_side_price = clob[opp_side].keys()
    if side == "buy":
        good_prices = [p for p in opp_side_price if p <= price]
    else:
        good_prices = [p for p in opp_side_price if p >= price]

    if not good_prices:
        return outstanding, matched_trades
    else:
        for good_price in sorted(good_prices):
            for opp_order in clob[opp_side][good_price]:
                opp_size = int(opp_order["size"])
                if outstanding == 0:
                    return 0, matched_trades
                if opp_size == 0:
                    continue
                if opp_size >= outstanding:
                    matched_trades.append({"pre-existing":int(opp_order["id"]),
                                           "latecomer": int(order["order id"]),
                                           "trade size": outstanding})
                    opp_size -= outstanding
                    return 0, matched_trades
                else:
                    outstanding -= opp_order["size"]
                    matched_trades.append({"pre-existing":int(opp_order["id"]),
                                           "latecomer": int(order["order id"]),
                                           "trade size": opp_size})
    

In [112]:
order = orders.iloc[10]
side, price, outstanding = order["side"], 6, int(order["size"])
opp_side = "buy" if side == "sell" else "sell"
opp_side_price = clob[opp_side].keys()
if side == "buy":
    good_prices = [p for p in opp_side_price if p <= price]
else:
    good_prices = [p for p in opp_side_price if p >= price]

good_price =sorted(good_prices)[0]
clob[opp_side][good_price]



[{'time': Timestamp('2024-09-30 20:26:27'), 'order id': 8, 'size': 142}]

In [90]:
def upload_clob_info(order, outstanding=""):
    if not outstanding:
        outstanding = order["size"]
    if type(outstanding) != int:
        outstanding = int(outstanding)
    price0 = float(order["price"])
    order_dic = {"time":order["timestamp"],
                 "order id": int(order["order id"]), 
                 "size": int(order["size"]),
                 }
    if price0 not in clob[order["side"]]:
        clob[order["side"]][price0] = [order_dic]
    else:
        clob[order["side"]][price0].append(order_dic)

def get_order_feature_at_the_price(target_feature, order="", price="", side="", opp=False):
    if not (order or (price and side)):
        print("Error: input either order or side&price")
        return None
    elif order:
        side, price = order["side"], float(order["price"])
    else:
        price = float(price)
    if opp:
        side = "buy" if side == "sell" else "sell"
    orders_at_the_price = clob[side][price]
    try:
        return [i[target_feature] for i in orders_at_the_price]
    except KeyError:
        print("Error: no such feature")
        return None
    
def upload_trade_info(trades):
    pass

In [4]:
# Print the updated CLOB in the loop
def pull_clob():
    pass

In [11]:
# Loop through all trades

for i in range(0, len(orders)):
    order = orders.iloc[i]
    outstanding, matched_trades = match_trade(order)
    if outstanding > 0:
        # Place the rest in the Order Book
        upload_clob_info(order, outstanding)
    
    
    # Print the updated records
    pull_clob()
        



In [91]:
# for experiment
for i in range(0, 10):
    order = orders.iloc[i]
    upload_clob_info(order)
clob